In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy.io import fits
from astropy import visualization as aviz
from astropy.table import Table
import subprocess

from adler.science.NoiseChisel import NoiseChisel

In [ ]:
infits = "../tests/data/ztf_Didymos-system-barycenter20065803_size256_20221201402616_000567_zr_c10_o_q1_scimrefdiffimg.fits.fz"  # runs with default noisechisel options
inhdu = 1

In [ ]:
# Inspect the fits image header data unit
hdu = fits.open(infits)
hdu.info()

In [ ]:
# Get image data and header
img = hdu[inhdu].data
hdr = hdu[inhdu].header

In [ ]:
# Look at the image
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

norm = aviz.ImageNormalize(img, interval=aviz.ZScaleInterval())
s1 = ax1.imshow(img, norm=norm, origin="lower")
cbar = plt.colorbar(s1)

plt.show()

In [ ]:
# Set up the adler noisechisel photometry class
nc = NoiseChisel(
    fits_file=infits,  # image to analyse
    i_hdu=inhdu,  # hdu index of image
)

In [ ]:
# inspect the noisechisel attributes, e.g. intermediate files to be saved
nc.__dict__

In [ ]:
# run just the noisechisel step
# N.B. that on jupyter RSP, gnuastro is not source installed. The command must be run within a conda environment (this is slow)

nc.noise_chisel()

# nc.noise_chisel(pre_cmd = ". /opt/lsst/software/stack/conda/etc/profile.d/conda.sh; conda run -n adler-dev-gnu ")

In [ ]:
# check that the noisechisel output files exist
! ls -altr $nc.file_nc $nc.file_check

In [ ]:
# inspect the noisechisel diagnostic check file
hdu_nc = fits.open(nc.file_check)
hdu_nc.info()

In [ ]:
# plot the noisechisel diagnostic check file
for i in range(len(hdu_nc) - 1):
    print(i + 1, hdu_nc[i + 1].name)
    img_nc = hdu_nc[i + 1].data
    hdr_nc = hdu_nc[i + 1].header

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1)
    ax1 = plt.subplot(gs[0, 0])
    # ax1 = plt.subplot(gs[0,0], projection = wcs) # use the original image wcs or load from another header of fout?

    norm = aviz.ImageNormalize(img_nc, interval=aviz.ZScaleInterval())
    s1 = ax1.imshow(img_nc, norm=norm, origin="lower")
    cbar = plt.colorbar(s1)

    plt.show()

In [ ]:
# inspect and plot the noisechisel detection fits file
hdu_nc = fits.open(nc.file_nc)
print(hdu_nc.info())

for i_plot in range(len(hdu_nc) - 1):
    print(i_plot + 1, hdu_nc[i_plot + 1].name)
    img_plot = hdu_nc[i_plot + 1].data
    hdr_plot = hdu_nc[i_plot + 1].header

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1)
    ax1 = plt.subplot(gs[0, 0])
    # ax1 = plt.subplot(gs[0,0], projection = wcs) # use the original image wcs or load from another header of fout?

    if i_plot == 0:
        norm = aviz.ImageNormalize(img_plot, interval=aviz.ZScaleInterval())
    else:
        norm = None
    s1 = ax1.imshow(img_plot, norm=norm, origin="lower")
    cbar = plt.colorbar(s1)

    plt.show()

In [ ]:
# now run the full NoiseChisel process, which includes segmenting the image and creating a catalogue of detections (with associated measurements)
# N.B. again, when running on RSP jupyter we must initialise conda (conda_start) and run within a conda env with gnuastro installed (conda_env) for the subprocess calls to work

df_cat = nc.run_noise_chisel()

# df_cat = nc.run_noise_chisel(conda_start=". /opt/lsst/software/stack/conda/etc/profile.d/conda.sh", conda_env="adler-dev-gnu")

In [ ]:
df_cat

In [ ]:
hdu_seg = fits.open(nc.file_seg)
print(hdu_seg.info())

In [ ]:
for i_plot in range(len(hdu_seg) - 1):
    print(i_plot + 1, hdu_seg[i_plot + 1].name)
    img_plot = hdu_seg[i_plot + 1].data
    hdr_plot = hdu_seg[i_plot + 1].header

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1)
    ax1 = plt.subplot(gs[0, 0])
    # ax1 = plt.subplot(gs[0,0], projection = wcs) # use the original image wcs or load from another header of fout?

    if i_plot == 0:
        norm = aviz.ImageNormalize(img_plot, interval=aviz.ZScaleInterval())
    else:
        norm = None
    s1 = ax1.imshow(img_plot, norm=norm, origin="lower")
    cbar = plt.colorbar(s1)

    plt.show()

In [ ]:
# f_cat = "ztf_Didymos-system-barycenter20065803_size256_20221201402616_000567_zr_c10_o_q1_scimrefdiffimg_detected_segmented_cat.fits"
# f_cat = "ztf_Didymos-system-barycenter20065803_size256_20221201402616_000567_zr_c10_o_q1_scimrefdiffimg_detected_cat.fits"
hdu_cat = fits.open(nc.file_cat)
print(hdu_cat.info())

In [ ]:
i_cat = 1  # objects
# i_cat = 2 # clumps
dat = hdu_cat[i_cat].data

In [ ]:
Table(dat)

In [ ]:
df_cat = Table(dat).to_pandas()
df_cat

In [ ]:
from matplotlib.patches import Ellipse

In [ ]:
i_plot = 1  # image
# i_plot = 3 # objects
# i_plot = 2 # clumps

print(i_plot, hdu_seg[i_plot].name)
img_plot = hdu_seg[i_plot].data
hdr_plot = hdu_seg[i_plot].header

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])
# ax1 = plt.subplot(gs[0,0], projection = wcs) # use the original image wcs or load from another header of fout?

if i_plot == 1:
    norm = aviz.ImageNormalize(img_plot, interval=aviz.ZScaleInterval())
else:
    norm = None
s1 = ax1.imshow(img_plot, norm=norm, origin="lower")
cbar = plt.colorbar(s1)

ax1.scatter(df_cat["X"], df_cat["Y"], c="r", s=1)

for i in range(len(df_cat)):
    ellipse = Ellipse(
        xy=(df_cat.iloc[i]["X"], df_cat.iloc[i]["Y"]),
        width=df_cat.iloc[i]["SEMI_MAJOR"] * 2.0,
        height=df_cat.iloc[i]["SEMI_MINOR"] * 2.0,
        angle=df_cat.iloc[i]["POSITION_ANGLE"],
        edgecolor="r",
        fc="None",
    )
    ax1.add_patch(ellipse)


plt.show()